In [1]:
pip install redshift_connector

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import redshift_connector
import pandas as pd

conn = redshift_connector.connect(
     host='redshift-cluster-1.cpop3m8tfd5a.us-east-1.redshift.amazonaws.com',
     database='olistdb',
     port=5439,
     user='awsuser',
     password='JldMVsOw&7xL'
  )

cursor = conn.cursor()

with conn.cursor() as cursor:
    cursor.execute("SELECT review_id, order_id, review_score, review_comment_message FROM \"olistdb\".\"public\".\"order_reviews\";")
    Review: pd.DataFrame = cursor.fetch_dataframe()

cursor.close()
conn.close()

In [3]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import RSLPStemmer
nltk.download('rslp')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [4]:
Review.dropna(subset = 'review_comment_message', inplace= True)

In [5]:
list_comment_message = Review['review_comment_message']

In [6]:
def remover_salto_de_linea(lista):
    return [re.sub('[\n\r]', ' ', r) for r in lista]
list_comment_message = remover_salto_de_linea(Review['review_comment_message'])
Review['review_comment_message'] = list_comment_message

In [7]:
def remover_links(lista):
    pattern = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return [re.sub(pattern, ' link ', r) for r in lista]
list_comment_message = remover_links(Review['review_comment_message'])
Review['review_comment_message'] = list_comment_message

In [8]:
def remover_fecha(lista):
    pattern = '([0-2][0-9]|(3)[0-1])(\/|\.)(((0)[0-9])|((1)[0-2]))(\/|\.)\d{2,4}'
    return [re.sub(pattern, ' date ', r) for r in lista]
list_comment_message = remover_fecha(Review['review_comment_message'])
Review['review_comment_message'] = list_comment_message

In [9]:
def remover_moneda(lista):
    pattern = '[R]{0,1}\$[ ]{0,}\d+(,|\.)\d+'
    return [re.sub(pattern, ' money ', r) for r in lista]
list_comment_message = remover_moneda(Review['review_comment_message'])
Review['review_comment_message'] = list_comment_message

In [10]:
def remover_numeros(lista):
    return [re.sub('[0-9]+', ' number ', r) for r in lista]
list_comment_message = remover_numeros(Review['review_comment_message'])
Review['review_comment_message'] = list_comment_message

In [11]:
def remover_caracter_especial(lista):
    return [re.sub('\W', ' ', r) for r in lista]
list_comment_message = remover_caracter_especial(Review['review_comment_message'])
Review['review_comment_message'] = list_comment_message

In [12]:
def remover_espacios(text_list):
    white_spaces = [re.sub('\s+', ' ', r) for r in text_list]
    white_spaces_end = [re.sub('[ \t]+$', '', r) for r in white_spaces]
    return white_spaces_end
list_comment_message = remover_espacios(Review['review_comment_message'])
Review['review_comment_message'] = list_comment_message

In [13]:
def remover_stopworlds(texto):
    return [c.lower() for c in texto.split() if c.lower() not in stopwords.words('portuguese')]

list_comment_message = [' '.join(remover_stopworlds(review)) for review in list_comment_message]
Review['review_comment_message'] = list_comment_message

In [14]:
def Busco_palabra_raiz(texto):
    stemmer=RSLPStemmer()
    return [stemmer.stem(c) for c in texto.split()]
    
list_comment_message = [' '.join(Busco_palabra_raiz(review)) for review in list_comment_message]
Review['review_comment_message']  = list_comment_message

In [15]:
count_vectorizer = CountVectorizer(max_features=500,max_df=0.7, min_df=7)

In [16]:
bag_of_words = count_vectorizer.fit_transform(list_comment_message)

In [17]:
feature_names = count_vectorizer.get_feature_names()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
Vectorizado_message = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

In [19]:
Review.reset_index(drop = True, inplace=True)

In [20]:
score0y1=[]
for row in Review['review_score']:
    if row in (4,5): score0y1.append(1)
    else: score0y1.append(0)
# la reescribo como la discreta, y entonces no tengo q cambiar nada de codigo abajo. 
Review['review_score']=score0y1

In [21]:
Dataframe_final = Vectorizado_message.join(Review['review_score'])

In [22]:
X_1_0 = Dataframe_final.drop('review_score', axis=1)
y_1_0 = Dataframe_final['review_score']

In [23]:
X_train_1_0, X_test_1_0, y_train_1_0, y_test_1_0 = train_test_split(X_1_0, y_1_0, test_size = 0.3, random_state = 42)

In [24]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 50, random_state=42)
rf.fit(X_train_1_0,y_train_1_0)
print("acc: ",rf.score(X_test_1_0,y_test_1_0))

acc:  0.8656072798886286


In [25]:
y_pred_rf=rf.predict(X_test_1_0)

In [26]:
df_y_pred_rf= pd.DataFrame(y_pred_rf)

In [27]:
df_y_pred_rf

,0
0,1
1,1
2,0
3,1
4,1
...,...
29446,1
29447,1
29448,1
29449,0


In [28]:
X_test_1_0_id = X_test_1_0.join(Review['review_id'])

In [29]:
X_test_1_0_id.reset_index(drop = True, inplace=True)
df_y_pred_rf.reset_index(drop = True, inplace=True)
df_rf_resultado = X_test_1_0_id.join(df_y_pred_rf[0])
df_rf_resultado = df_rf_resultado[[0, 'review_id']]

In [30]:
df_rf_resultado

,0,review_id
0,1,ca066ddee5978b04ecbd8b7155b21291
1,1,0e1059a405261feda792621aa2d0699a
2,0,99e00d2ab2ed39968f74115fbaabc561
3,1,4dbb0787588f28909efbc886f439aa59
4,1,c5489f52b234482f5e4a784c518b33c1
...,...,...
29446,1,4b6d8173b31fd14c1c4b826973c31568
29447,1,a5001772ac74d2fe07a2dcca336077c8
29448,1,9e5a62ba153267a922867dc8fd3cb287
29449,0,30bbf834f9d9b35e15824033e896b88c


In [31]:
conn = redshift_connector.connect(
     host='redshift-cluster-1.cpop3m8tfd5a.us-east-1.redshift.amazonaws.com',
     database='olistdb',
     port=5439,
     user='awsuser',
     password='JldMVsOw&7xL'
  )
conn.autocommit = True
cursor = conn.cursor()

with conn.cursor() as cursor:
    cursor.execute('DROP TABLE IF EXISTS "public"."ml_rf_resultados";')
    cursor.execute('CREATE TABLE "public"."ml_rf_resultados"(prediccion varchar(255), review_id character varying(256));')
    cursor.execute('SELECT * FROM "public"."ml_rf_resultados";')
    result = cursor.fetchall()
    
cursor.close()
conn.close()
print(result)
###
#with conn.cursor() as cursor:
    #cursor.write_dataframe(df_rf_resultado, "ML_rf_resultados")
    #cursor.execute('SELECT * FROM "olistdb"."public"."ML_rf_resultados";')
    #result: pd.DataFrame = cursor.fetch_dataframe()

()


In [36]:
conn = redshift_connector.connect(
     host='redshift-cluster-1.cpop3m8tfd5a.us-east-1.redshift.amazonaws.com',
     database='olistdb',
     port=5439,
     user='awsuser',
     password='JldMVsOw&7xL'
  )
conn.autocommit = True
cursor = conn.cursor()

with conn.cursor() as cursor:
    cursor.write_dataframe(df_rf_resultado, 'ml_rf_resultados')
    cursor.execute('SELECT * FROM "public"."ml_rf_resultados";')
    result = cursor.fetch_dataframe()

cursor.close()
conn.close()
print(result)

      prediccion                         review_id
0              1  ca066ddee5978b04ecbd8b7155b21291
1              0  99e00d2ab2ed39968f74115fbaabc561
2              1  c5489f52b234482f5e4a784c518b33c1
3              1  f6a499caf9f7778cb7144a4b05ebfb50
4              0  c2fbe0703e013c3fe9d017cb9425415c
...          ...                               ...
29446          1  590036ec3cb186b264cf6338059fdcfe
29447          1  a961869480b9217c62542f7a512caf0d
29448          1  fc5dd4e8b72d4693a5ad73cb6d8d0fee
29449          1  4b6d8173b31fd14c1c4b826973c31568
29450          1  9e5a62ba153267a922867dc8fd3cb287

[29451 rows x 2 columns]
